<summary>
    <font size="5" color="gray"><b> CONVENIO CENACE-UNISON 2023-2025
 </b></font>
</summary><br>
<summary>
    <font size="4" color="orange"><b> DISEÑO E IMPLEMENTACIÓN DE UNA METODOLOGÍA PARA PRONÓSTICO DE DEMANDA DE ENERGÍA A CORTO PLAZO CON MANEJO DE DATOS BAJO INCERTIDUMBRE
 </b></font>
</summary>

---

<summary>
    <font size="8" color="gray"><b> Libreta JNB0: Preprocesamiento
 </b></font>
</summary>

---
    
OLIVIA CAROLINA GUTÚ OCAMPO

REYNA YANET HERNÁNDEZ MADA

JULIO WAISSMAN VILANOVA (Responsable del proyecto)


---


<summary>
    <font size="4" color="orange"><b> INPUTS
 </b></font>
</summary>

<summary>
    <font size="2" color="gray"><b> Dataset 1:
 </b></font>
</summary>


Fuente CENACE
* Fecha en formato yyy-mm-dd hh:00:00, desde 2007-01-01 a la fecha de actualización.

* 24 variables de demanda de energía (MW), datos reales-medidos de la demanda de carga de energía eléctrica en la Zona de Gerencia del Noroeste por cada hora del día compuesta por zonas de carga, a saber, Caborca, Nogales, Hermosillo, Guaymas, Cd. Obregón, Navojoa, Los Mochis, Guasave, Culiacán y Mazatlán, cargas fuera del consumo de las zonas de carga descritas anteriormente, el consumo de las centrales y las pérdidas de energía que existen en la red.

* 10 variables de calendario: lunes festivo, lunes después de festivo, Semana Santa, 1.° de mayo, 10 de mayo, 16 de septiembre, 2 de noviembre, día antes de Navidad o Año Nuevo, Navidad o Año Nuevo, día después de Navidad o Año Nuevo. 

Fuente Comisión Nacional del Agua (CONAGUA):
* Fecha en formato yyyy-mm-dd, desde 2007-01-01 a la fecha de actualización.

* 14 variables meteorológicas (°C and mm/h), datos reales-medidos de la temperatura máxima y mínima diaria en Caborca, Hermosillo, Ciudad Obregón, Los Mochis y Culiacán. Precipitación diaria en Hermosillo, Ciudad Obregón, Los Mochis y Culiacán.

--- 
<summary>
    <font size="4" color="orange"><b> CHECK POINTS
 </b></font>
</summary>


* Antes de exportar el dataset normalizado se verifica que no tenga valores faltantes.

---
<summary>
    <font size="4" color="orange"><b> OUTPUTS (INTERNAL INPUTS)
 </b></font>
</summary>

Archivo .csv de serie de tiempo exportado al path **'./internal_inputs/GCRNO_complete_silver_[fecha_de_hoy].csv'** (entrada de la libreta YOO-JNB1) con índice **Date_time** y features:

* **Energy_Demand**

* **Hour**

* **Day**

* **Month**

* **Tmax-Cab**

* **Tmax-Hmo** 

* **Tmax-Obr**

* **Tmax-Lmo** 

* **Tmax-Cul** 

* **Tmin-Cab** 

* **Tmin-Hmo** 

* **Tmin-Obr**

* **Tmin-Lmo** 

* **Tmin-Cul**

* **Prec_Hmo_Mm** 

* **Prec_Obr_Mm** 

* **Prec_Lmo_Mm**

* **Prec_Cul_Mm**

* **Monday_Holiday**

* **Tuesday_Aft_Hol**

* **Easter_week**

* **May_1s** 

* **May_10t**

* **Sept_16** 

* **Nov_2nd**

* **Before_Christmas_NY**

* **Christmas_NY**

* **After_Christmas_NY**

## 1. Importing libraries

In [1]:
# Do not print warnings
import warnings
warnings.filterwarnings('ignore')

# Basic libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Auxiliar libraries
import os
from datetime import date
from pathlib import Path  

## 2. Download raw dataset

In [2]:
# Download CENACE Dataset 1
df_dataset_1 = pd.read_excel('./inputs/Dataset _GCRNO.xlsx')

## 3. Setting multivariate time series

In [3]:
# Transposing hours columns from the original dataframe into rows
df_gcrno = df_dataset_1.melt(
    id_vars = ['FECHA'],
    value_vars = [f'DEM_GCRNO_H{i}' for i in range(24)],
    var_name = 'HORA',
    value_name = 'DEMANDA'
).replace(
    {f'DEM_GCRNO_H{i}': i for i in range(24)}
)

In [4]:
# Creating Day, Hour and Month columns
df_gcrno['Date'] = pd.to_datetime(df_gcrno['FECHA'], format='%d/%m/%Y')
df_gcrno.index = df_gcrno.FECHA + pd.to_timedelta(df_gcrno.HORA, unit='h')
df_gcrno.sort_index(inplace=True)
df_gcrno.drop(columns=['Date'], inplace=True)
df_gcrno = df_gcrno.asfreq('h', method='pad')
df_gcrno['Date_time'] = df_gcrno.index
df_gcrno['Day'] = df_gcrno.index.weekday
df_gcrno['Month'] = df_gcrno.index.month

# Updating column titles in consumo
df_gcrno.rename(columns={'FECHA':'Date', 'HORA': 'Hour', 'DEMANDA':'Energy_Demand'}, inplace = True)
df_gcrno.head(2)

,Date,Hour,Energy_Demand,Date_time,Day,Month
2007-01-01 00:00:00,2007-01-01,0,1394,2007-01-01 00:00:00,0,1
2007-01-01 01:00:00,2007-01-01,1,1297,2007-01-01 01:00:00,0,1


In [5]:
# Uniformize columns names before merge
# Adding columns of exogenous variables
df_exogenous = df_dataset_1[['FECHA',
                             'TMAX-CAB', 
                             'TMAX-HMO', 
                             'TMAX-OBR', 
                             'TMAX-LMO', 
                             'TMAX-CUL', 
                             'TMIN-CAB',
                             'TMIN-HMO', 
                             'TMIN-OBR', 
                             'TMIN-LMO', 
                             'TMIN-CUL', 
                             'PREC_HMO_MM',
                             'PREC_OBR_MM', 
                             'PREC_LMO_MM', 
                             'PREC_CUL_MM', 
                             'LUNES_FESTIVO',
                             'MARTES_POSTFESTIVO', 
                             'SEMANA_SANTA', 
                             '1_MAYO', 
                             '10_MAYO', 
                             '16_SEP',
                             '2_NOV.', 
                             'PRE-NAVIDAD_Y_NEW_YEAR', 
                             'NAVIDAD_Y_NEW_YEAR',
                             'POST-NAVIDAD_Y_NEW_YEAR'
                            ]]
df_exogenous.rename(columns = {'FECHA':'Date',
                               'TMAX-CAB':'Tmax-Cab', 
                               'TMAX-HMO':'Tmax-Hmo', 
                               'TMAX-OBR':'Tmax-Obr', 
                               'TMAX-LMO':'Tmax-Lmo', 
                               'TMAX-CUL':'Tmax-Cul', 
                               'TMIN-CAB':'Tmin-Cab',
                               'TMIN-HMO':'Tmin-Hmo', 
                               'TMIN-OBR':'Tmin-Obr', 
                               'TMIN-LMO':'Tmin-Lmo', 
                               'TMIN-CUL':'Tmin-Cul', 
                               'PREC_HMO_MM':'Prec_Hmo_Mm',
                               'PREC_OBR_MM':'Prec_Obr_Mm', 
                               'PREC_LMO_MM':'Prec_Lmo_Mm', 
                               'PREC_CUL_MM':'Prec_Cul_Mm', 
                               'LUNES_FESTIVO':'Monday_Holiday',
                               'MARTES_POSTFESTIVO':'Tuesday_Aft_Hol', 
                               'SEMANA_SANTA': 'Easter_week', 
                               '1_MAYO':'May_1s', 
                               '10_MAYO':'May_10t', 
                               '16_SEP': 'Sept_16',
                               '2_NOV.':'Nov_2nd', 
                               'PRE-NAVIDAD_Y_NEW_YEAR':'Before_Christmas_NY', 
                               'NAVIDAD_Y_NEW_YEAR':'Christmas_NY',
                               'POST-NAVIDAD_Y_NEW_YEAR':'After_Christmas_NY'
                              }, inplace = True)
df_exogenous['Date']= pd.to_datetime(df_exogenous['Date'], format='%d/%m/%Y')
df_exogenous.head(2)

,Date,Tmax-Cab,Tmax-Hmo,Tmax-Obr,Tmax-Lmo,Tmax-Cul,Tmin-Cab,Tmin-Hmo,Tmin-Obr,Tmin-Lmo,...,Monday_Holiday,Tuesday_Aft_Hol,Easter_week,May_1s,May_10t,Sept_16,Nov_2nd,Before_Christmas_NY,Christmas_NY,After_Christmas_NY
0,2007-01-01,21.0,22.0,25.0,30.0,29.0,2.0,9.0,8.0,10.0,...,0,0,0,0,0,0,0,0,1,0
1,2007-01-02,21.0,22.0,22.0,22.0,27.0,2.0,7.0,7.0,11.0,...,0,0,0,0,0,0,0,0,0,1


In [6]:
# Merging endogenous and exogenous dataframes
df_GCRNO_complete =  pd.merge(df_gcrno, df_exogenous, on='Date', how='left')

In [7]:
# 'Date_time' as index: setting hourly time series 
df_GCRNO_complete.set_index('Date_time', inplace = True)
df_GCRNO_complete = df_GCRNO_complete.asfreq('h')

# Remove 'Date' column
df_GCRNO_complete.drop('Date', inplace=True, axis=1)
df_GCRNO_complete.tail(2)

,Hour,Energy_Demand,Day,Month,Tmax-Cab,Tmax-Hmo,Tmax-Obr,Tmax-Lmo,Tmax-Cul,Tmin-Cab,...,Monday_Holiday,Tuesday_Aft_Hol,Easter_week,May_1s,May_10t,Sept_16,Nov_2nd,Before_Christmas_NY,Christmas_NY,After_Christmas_NY
Date_time,,,,,,,,,,,,,,,,,,,,,
2022-05-12 22:00:00,22,3777,3,5,34.0,37.0,38.0,36.0,40.0,15.0,...,0,0,0,0,0,0,0,0,0,0
2022-05-12 23:00:00,23,3796,3,5,34.0,37.0,38.0,36.0,40.0,15.0,...,0,0,0,0,0,0,0,0,0,0


## 4. Check point: verifying missing data

In [8]:
df_GCRNO_complete.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 134664 entries, 2007-01-01 00:00:00 to 2022-05-12 23:00:00
Freq: H
Data columns (total 28 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Hour                 134664 non-null  int64  
 1   Energy_Demand        134664 non-null  int64  
 2   Day                  134664 non-null  int32  
 3   Month                134664 non-null  int32  
 4   Tmax-Cab             134664 non-null  float64
 5   Tmax-Hmo             134664 non-null  float64
 6   Tmax-Obr             134664 non-null  float64
 7   Tmax-Lmo             134664 non-null  float64
 8   Tmax-Cul             134664 non-null  float64
 9   Tmin-Cab             134664 non-null  float64
 10  Tmin-Hmo             134664 non-null  float64
 11  Tmin-Obr             134664 non-null  float64
 12  Tmin-Lmo             134664 non-null  float64
 13  Tmin-Cul             134664 non-null  float64
 14  Prec_Hmo_Mm          13466

## 5. Exporting output file

In [9]:
# Preparing the name of the file with the date of creation
today = date.today().strftime("%Y-%m-%d")
filepath = Path('./internal_inputs/GCRNO_complete_silver_'+ today + '.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)  

In [10]:
# Exporting df_GCRNO_complete datafame
df_GCRNO_complete.to_csv(filepath, index = True)

<summary>
    <font size="4" color="gray"> Maestría en Ciencia de Datos | Universidad de Sonora </font>
</summary>
<font size="1" color="gray"> Blvd. Luis Encinas y Rosales s/n Col. Centro. Edificio 3K1 planta baja C.P. 83000, Hermosillo, Sonora, México </font>
<font size="1" color="gray"> mcd@unison.mx </font>
<font size="1" color="gray"> Tel: +52 (662) 259 2155  </font>